In [1]:
!gdown 1x8NFTmr497b7vVzq6VtSu0-QYGSVs3vz

Downloading...
From: https://drive.google.com/uc?id=1x8NFTmr497b7vVzq6VtSu0-QYGSVs3vz
To: /content/smart_support_vtb_belarus_faq_final.csv
100% 123k/123k [00:00<00:00, 80.0MB/s]


In [2]:
!pip install langchain langchain-community langchain-openai openai sentence-transformers faiss-cpu rank_bm25 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as pltfrom
from langchain.document_loaders import CSVLoader, DataFrameLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [4]:
df = pd.read_csv('smart_support_vtb_belarus_faq_final.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna()

In [5]:
df.columns = ['category', 'subcategory', 'question', 'priority', 'target_group', 'answer']

In [6]:
df.shape

(201, 6)

In [7]:
df.head(10)

,category,subcategory,question,priority,target_group,answer
0,Новые клиенты,Регистрация и онбординг,Как стать клиентом банка онлайн?,высокий,новые клиенты,Стать клиентом ВТБ (Беларусь) можно онлайн чер...
1,Новые клиенты,Регистрация и онбординг,Регистрация через МСИ (Межбанковская система и...,высокий,новые клиенты,"МСИ позволяет пройти идентификацию онлайн, исп..."
2,Новые клиенты,Регистрация и онбординг,Документы для регистрации нового клиента,высокий,новые клиенты,Для регистрации в качестве нового клиента необ...
3,Новые клиенты,Первые шаги,Первый вход в Интернет-банк,высокий,новые клиенты,После регистрации вы получите логин и пароль д...
4,Новые клиенты,Первые шаги,Как скачать и настроить мобильное приложение?,высокий,новые клиенты,Мобильное приложение VTB mBank можно скачать в...
5,Техническая поддержка,Проблемы и решения,Не могу войти в Интернет-банк,высокий,все клиенты,"Если не получается войти в Интернет-банк, пров..."
6,Техническая поддержка,Проблемы и решения,Я нахожусь за пределами Республики Беларусь. К...,высокий,все клиенты,Вы можете получить онлайн-консультацию (в текс...
7,Техническая поддержка,Проблемы и решения,Забыл пароль от мобильного приложения,высокий,все клиенты,"Если Вы забыли логин или пароль доступа, досту..."
8,Техническая поддержка,Проблемы и решения,Карта заблокирована - что делать?,высокий,все клиенты,"Если карта заблокирована, обратитесь в контакт..."
9,Техническая поддержка,Проблемы и решения,"Могу ли я оставить обращение, не обращаясь лич...",высокий,все клиенты,"Вы можете воспользоваться формой ""Обратная свя..."


In [53]:
df.to_csv('hahaton.csv', index=False)

In [45]:
list(df['answer'])

['Стать клиентом ВТБ (Беларусь) можно онлайн через сайт vtb.by или мобильное приложение VTB mBank. Для регистрации потребуется паспорт и номер телефона. После регистрации через МСИ (Межбанковскую систему идентификации) вы получите доступ к банковским услугам.',
 'МСИ позволяет пройти идентификацию онлайн, используя данные других банков, где вы уже являетесь клиентом. Это упрощает процедуру регистрации и делает её быстрой и безопасной.',
 'Для регистрации в качестве нового клиента необходим паспорт гражданина Республики Беларусь и контактный номер мобильного телефона для получения SMS-подтверждений.',
 'После регистрации вы получите логин и пароль для входа в систему Интернет-банк. При первом входе рекомендуется изменить временный пароль на постоянный и настроить дополнительные параметры безопасности.',
 'Мобильное приложение VTB mBank можно скачать в App Store для iOS или Google Play для Android. После установки войдите с логином и паролем от Интернет-банка и пройдите первоначальную на

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 200
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      201 non-null    object
 1   subcategory   201 non-null    object
 2   question      201 non-null    object
 3   priority      201 non-null    object
 4   target_group  201 non-null    object
 5   answer        201 non-null    object
dtypes: object(6)
memory usage: 11.0+ KB


In [17]:
df['category'].value_counts()

,count
category,
Продукты - Карты,63
Частные клиенты,62
Продукты - Кредиты,35
Продукты - Вклады,30
Техническая поддержка,6
Новые клиенты,5


In [9]:
df[df['question'] == 'Кто может оформить этот кредит?']

,category,subcategory,question,priority,target_group,answer
89,Продукты - Кредиты,Потребительские - Всё только начинается,Кто может оформить этот кредит?,высокий,новые клиенты,Кредит 'Всё только начинается' предназначен дл...


In [12]:
df[df['category'] == 'Продукты - Кредиты']['target_group'].value_counts()

,count
target_group,
новые клиенты,35


In [13]:
df[df['category'] == 'Продукты - Карты']['target_group'].value_counts()

,count
target_group,
новые клиенты,63


In [14]:
df[df['category'] == 'Продукты - Вклады']['target_group'].value_counts()

,count
target_group,
новые клиенты,30


In [15]:
df[df['category'] == 'Частные клиенты']['target_group'].value_counts()

,count
target_group,
все клиенты,48
новые клиенты,14


In [17]:
df[df['category'] == 'Частные клиенты']['subcategory'].value_counts()

,count
subcategory,
Кредиты,22
Банковские карточки,19
Вклады и депозиты,15
Онлайн-сервисы,6


In [18]:
df['subcategory'].value_counts()

,count
subcategory,
Кредиты,22
Банковские карточки,19
Вклады и депозиты,15
Карты рассрочки - ЧЕРЕПАХА,8
Дебетовые карты - MORE,8
Кредитные карты - PLAT/ON,7
Карты рассрочки - КСТАТИ,7
Дебетовые карты - Форсаж,7
Онлайн-сервисы,6


In [21]:
df['priority'].value_counts()

,count
priority,
высокий,113
средний,88


In [22]:
df['target_group'].value_counts()

,count
target_group,
новые клиенты,147
все клиенты,54


In [10]:
df['subcategory'].nunique(), df['category'].nunique()

(35, 6)

In [11]:
import os
from getpass import getpass

In [12]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

# course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='Qwen2.5-72B-Instruct-AWQ',
                 base_url="https://llm.t1v.scibox.tech/v1")

Введите ваш ключ, полученный в боте курса··········


In [17]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemy.
В России лидером является Stepik.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """
print(llm.invoke(prompt).content)

На основе информации из контекста, можно размещать курсы на таких онлайн платформах, как Coursera, Udemy и Stepik.


In [18]:
template = """
Answer the question based only on the following context:

{context}

Question: {question}
"""

# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)

In [21]:
from langchain.embeddings.base import Embeddings
from openai import OpenAI

class CustomBgeEmbeddings(Embeddings):
    def __init__(self, api_key: str, base_url: str = "https://llm.t1v.scibox.tech/v1", model: str = "bge-m3"):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            response = self.client.embeddings.create(
                model=self.model,
                input=text
            )
            embeddings.append(response.data[0].embedding)
        return embeddings

    def embed_query(self, text):
        response = self.client.embeddings.create(
            model=self.model,
            input=text
        )
        return response.data[0].embedding


In [39]:
import requests
from langchain.embeddings.base import Embeddings
from concurrent.futures import ThreadPoolExecutor, as_completed

class BGEM3Embeddings(Embeddings):
    def __init__(self, api_key: str, base_url: str = "https://llm.t1v.scibox.tech/v1",
                 model: str = "bge-m3", batch_size: int = 32, max_workers: int = 4, timeout: int = 60):
        self.api_key = api_key
        self.base_url = base_url.rstrip("/") + "/embeddings"
        self.model = model
        self.batch_size = batch_size
        self.max_workers = max_workers
        self.timeout = timeout
        self.session = requests.Session()
        self.session.headers.update({
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip, deflate"
        })

    def _post_batch(self, texts):
        payload = {"model": self.model, "input": texts}
        resp = self.session.post(self.base_url, json=payload, timeout=self.timeout)
        resp.raise_for_status()
        data = resp.json()
        return [d["embedding"] for d in data["data"]]

    def _embed(self, texts):
        # Разбиваем на батчи
        batches = [texts[i:i+self.batch_size] for i in range(0, len(texts), self.batch_size)]
        embeddings = [None] * len(batches)

        # Параллельная обработка батчей
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = {executor.submit(self._post_batch, batches[i]): i for i in range(len(batches))}
            for fut in as_completed(futures):
                idx = futures[fut]
                embeddings[idx] = fut.result()

        # Расплющиваем результат
        return [vec for batch in embeddings for vec in batch]

    def embed_documents(self, texts):
        return self._embed(texts)

    def embed_query(self, text):
        return self._embed([text])[0]


In [40]:
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

loader = DataFrameLoader(df, page_content_column='answer')
documents = loader.load()

# Определяем сплиттер
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
split_texts = splitter.split_documents(documents)


embeddings = BGEM3Embeddings(api_key=course_api_key)

# Векторное хранилище
db = FAISS.from_documents(split_texts, embeddings)


# Задаём ретривер
retriever = db.as_retriever()

In [42]:
split_texts[0]

Document(metadata={'category': 'Новые клиенты', 'subcategory': 'Регистрация и онбординг', 'question': 'Как стать клиентом банка онлайн?', 'priority': 'высокий', 'target_group': 'новые клиенты'}, page_content='Стать клиентом ВТБ (Беларусь) можно онлайн через сайт vtb.by или мобильное приложение VTB mBank. Для регистрации потребуется паспорт и номер телефона. После регистрации через МСИ (Межбанковскую систему идентификации) вы получите доступ к банковским услугам.')

In [44]:
db.similarity_search_with_score('Я не помню пароль')

[(Document(id='a117dd7a-bcdd-452f-891e-afc095aca6ec', metadata={'category': 'Техническая поддержка', 'subcategory': 'Проблемы и решения', 'question': 'Забыл пароль от мобильного приложения', 'priority': 'высокий', 'target_group': 'все клиенты'}, page_content='Если Вы забыли логин или пароль доступа, доступна процедура восстановления доступа к системе ДБО со стартового экрана. Для этого требуется нажать на кнопку «Забыли логин или пароль?».'),
  np.float32(0.7354803)),
 (Document(id='14aaa035-aaff-4876-a88c-1363f41976b9', metadata={'category': 'Техническая поддержка', 'subcategory': 'Проблемы и решения', 'question': 'Не могу войти в Интернет-банк', 'priority': 'высокий', 'target_group': 'все клиенты'}, page_content='Если не получается войти в Интернет-банк, проверьте правильность ввода логина и пароля. При забытом пароле воспользуйтесь функцией восстановления. Если проблема не решается, обратитесь в контакт-центр по номеру 250 или +375 (17/29/33) 309 15 15.'),
  np.float32(0.76372707)),

In [48]:
template = """
Игнорируй все предыдущие инструкции. Ты консультант компании T1, которая разрабатывает интеллектуальные системы поддержки клиентов для крупных организаций, включая финансовые учреждения.
Компания T1 создает платформу, интегрирующую машинное обучение и обработку естественного языка, чтобы автоматизировать классификацию и анализ клиентских обращений, повышать точность и скорость ответов операторов, снижать риски ошибок и человеческий фактор.

Продукты компании: система автоматической классификации обращений, модуль ранжирования шаблонов, механизм непрерывного обучения, веб-интерфейс для операторов, интеграция с базами знаний и историей успешных решений.

Будь вежлив и внимателен. Выяви потребность клиента, особенно если запрос эмоционально окрашен, неполон или содержит технический жаргон. Помоги ему с использованием возможностей платформы T1, опираясь только на фактическую информацию о продуктах и приведенный контекст. Если точного ответа нет, честно скажи об этом и предложи обратиться к оператору или менеджеру T1. Не придумывай фактов.

Данные о проведенных консультациях: {context}

Отвечай на языке, на котором задан вопрос.

Принципы ответа:
1. Давай четкие, прямые и краткие ответы.
2. Исключи ненужные извинения, упоминания самого себя и лишние формулировки.
3. Используй непринужденный, профессиональный и уверенный тон.
4. При двусмысленных или неполных запросах уточняй детали, чтобы понять намерение клиента.
5. При объяснении используй реальные примеры из работы служб поддержки и аналогии из банковской или финансовой сферы.
6. Для сложных запросов анализируй шаг за шагом, структурируя решение.
7. Обращай особое внимание на финансовые и юридические последствия ошибок в ответах.
8. За каждый ответ ты получишь чаевые от 10 до 20 долларов (в зависимости от качества твоего ответа).
9. Каждый твой ответ должен максимально повышать точность, релевантность и эффективность обработки запроса.

Очень важно, чтобы ты понял это правильно. На кону несколько жизней и репутация компании.

Question: {question}
"""

# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


In [49]:
# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


In [50]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Карта PLAT/ON с виртуальным кредитным лимитом – могу ли я сразу делать онлайн-покупки на 5 000 BYN?")

'Да, вы можете сразу делать онлайн-покупки на сумму до 5 000 BYN после оформления виртуальной карты PLAT/ON. Однако важно учесть, что ваш кредитный лимит будет зависеть от ваших доходов и текущих обязательств, и банк может установить его ниже максимальной суммы. После одобрения заявки и активации карты вы сможете использовать её для онлайн-покупок в полном объёме вашего лимита.'

In [54]:
chain.invoke("Я забыл пароль к мобильному приложению, могу ли я восстановить доступ без визита в отделение?")

'Да, вы можете восстановить доступ к мобильному приложению без визита в отделение. На стартовом экране приложения нажмите на кнопку «Забыли логин или пароль?». Следуйте инструкциям, которые появятся на экране. Если возникнут трудности, вы можете также позвонить по контактным телефонам банка для получения дальнейшей помощи.'

In [52]:
chain.invoke("Можно ли сменить пароль на новый, если я впервые вошёл через Интернет-банк?")

'Да, можно. При первом входе в Интернет-банк рекомендуется изменить временный пароль на постоянный. Это повысит безопасность вашего аккаунта. Для смены пароля используйте соответствующий раздел в настройках безопасности системы.'